In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Annotated
import operator
from langgraph.types import CachePolicy
from langgraph.cache.memory import InMemoryCache
from datetime import datetime
from typing import Literal
from langgraph.types import Send

In [ ]:
from typing import Union

def update_function(old,new):
    return old + new

class State(TypedDict):
    words: list[str]
    output: Annotated[list[dict[str, Union[str, int]]], operator.add]


graph_builder = StateGraph(
    State
    )

def node_one(state: State):
    print(f"I want to count {len(state['words'])} words in my state.")

def node_two(word: State):
    return {
        "output": [
            {
                "word": word,
                "letters": len(word)
            }
        ]
    }
       




In [ ]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)

def dispatcher(state: State):
    return [Send("node_two", word) for word in state["words"]]


graph_builder.add_edge(START, "node_one")
graph_builder.add_conditional_edges("node_one", dispatcher, ["node_two"])
graph_builder.add_edge("node_two", END)

#def decide_path(state: State) -> Literal["node_three", "node_four"]:
#    if state["seed"] % 2 == 0:
#        return "node_three"
#    else:
#        return "node_four"

def decide_path(state: State):
    return state["seed"] % 2 == 0

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_conditional_edges("node_two", decide_path, {True: "node_three", False: "node_four", "hello": END})
graph_builder.add_edge("node_four", END)

graph = graph_builder.compile()

graph.invoke(
    {
        "words": ["hello","world", "how", "are", "you", "doing"]
    }
)

                        +-----------+                    
                        | __start__ |                    
                        +-----------+                    
                               *                         
                               *                         
                               *                         
                         +----------+                    
                         | node_one |                    
                         +----------+                    
                               *                         
                               *                         
                               *                         
                         +----------+                    
                         | node_two |.                   
                      ...+----------+ ....               
                 .....         .          ....           
              ...              .              ....       
           ...

In [ ]:
import time 
graph = graph_builder.compile(cache=InMemoryCache())

print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)
print(graph.invoke({}))
time.sleep(5)

